# Accuracy-based BO example

In [7]:
include("src/BayesianCausalityOptimization.jl");

using .BayesianCausalityOptimization

valid_pairs = setdiff(1:100,[52,53,54,55,71])
tuebingen_data = BayesianCausalityOptimization.Datasets.tuebingen(valid_pairs);
#pair_ids = ["001","007","016","025","034","049","064","072","073","078","086","087","088","096","100"]
#selected_data = create_data(tuebingen_data, pair_ids, 1000);

T_max = 10
Rs = 5:5
EPOCHS = 10
M = 2
P = 10
n_iterations = 10

optimizer, optimum = bayesian_optimization(tuebingen_data, T_max;Rs=Rs, EPOCHS=EPOCHS, M=M, P=P, 
                                            n_iterations=n_iterations)

# Likelihood-based BO

In [1]:
include("src/BayesianCausalityOptimization.jl");
include("src/BayesianCausality.jl");
include("src/Datasets.jl");
include("src/Misc.jl");

using .Misc, .BayesianCausality, .BayesianCausalityOptimization
using .Datasets
using Statistics, Distributions
using SpecialFunctions, LinearAlgebra
import Base.Iterators: product
import Random: randperm, seed!
using BayesOpt
import Base.Iterators: enumerate
import Base.Filesystem: mkpath

SEED = 100
seed!(SEED)
RESULTS_PATH = "./results/tuebingen/vb-hp-opt"
PARAMS_PATH = "./params";

In [2]:
valid_pairs = setdiff(1:100,[52,53,54,55,71])
tuebingen_data = BayesianCausalityOptimization.Datasets.tuebingen(valid_pairs);

┌ Warning: readtable is deprecated, use CSV.read from the CSV package instead
│   caller = ip:0x0
└ @ Core :-1


In [3]:
function prepare_empty_parameter_dictionary(Rs::UnitRange{Int64}, data::Array{Dict{Symbol,Any},1})
    DIRECTION = ['>','<', '^']
    resulting_parameters = Dict()
    likelihoods = Dict()
    for (id, pair) ∈ enumerate(data)
        resulting_parameters[pair[:id]] = Dict()
        likelihoods[pair[:id]] = Dict()
        for R in Rs
            resulting_parameters[pair[:id]][R] = Dict()
            likelihoods[pair[:id]][R] = Dict()
            for direction in DIRECTION
                resulting_parameters[pair[:id]][R][direction] = Dict()
                likelihoods[pair[:id]][R][direction] = Dict()
            end
        end
    end
    return resulting_parameters, likelihoods
end

function bayesian_optimization_likelihood(xⁿ, yⁿ; R, EPOCHS, M, P, n_iterations, m₁, m₂)
    config = ConfigParameters()        # calls initialize_parameters_to_default of the C API
    config.n_iterations = n_iterations;
    config.verbose_level = 0
    set_kernel!(config, "kMaternARD5")  # calls set_kernel of the C API
    config.sc_type = SC_MAP;
    f((γ, λ₁, λ₂, a₁, a₂, b₁, b₂)) = -v_bayes([xⁿ yⁿ],R;M=M,γ=exp(γ),m₁=m₁,m₂=m₂,λ₁=exp(λ₁),λ₂=exp(λ₂),
        a₁=exp(a₁),a₂=exp(a₂),b₁=exp(b₁),b₂=exp(b₂),EPOCHS=EPOCHS)[1][end]
    param_keys = ["γ", "λ₁", "λ₂", "a₁", "a₂", "b₁", "b₂"]
    lowerbound = [log(.0001), log(.0001), log(.0001), log(.0001), log(.0001), log(.0001), log(.0001)]; 
    upperbound = [log(10000.), log(10000.), log(10000.), log(10000.), log(10000.), log(10000.), log(10000.)];
    @time optimizer, optimum = bayes_optimization(f, lowerbound, upperbound, config)
    return Dict(zip(param_keys, optimizer)), optimum
    end;
                    
function vb_hyperparameter_optimization(data,T_max=Inf; name="hp_opt_likelihood", M=1, P=1, Rs=1:1, m₁=0.0, m₂=0.0,
        EPOCHS=1, warm_start_type="free", n_iterations=200, norm=true)
    mkpath("$RESULTS_PATH/$(name)_$(warm_start_type)")
    DIRECTION = ['>','<', '^']
    parameters, likelihoods = prepare_empty_parameter_dictionary(Rs, data)
    for (id,pair) ∈ enumerate(data)
    	print(pair[:id],"...\t\t\t")
        T = length(pair[:X])
        perm = randperm(T)[1:Int(min(T,T_max))]
        x, y = pair[:X][perm], pair[:Y][perm]
        for R in Rs
            println(" ", R)
            xⁿ = norm ? (x .- mean(x)) / std(x) : x
            yⁿ = norm ? (y .- mean(y)) / std(y) : y
            parameters[pair[:id]][R]['^'], likelihoods[pair[:id]][R]['^'] = bayesian_optimization_likelihood(xⁿ, yⁿ; 
                R=R, EPOCHS=EPOCHS, M=1, P=P, n_iterations=n_iterations, m₁=m₁, m₂=m₂)
            parameters[pair[:id]][R]['>'], likelihoods[pair[:id]][R]['>'] = bayesian_optimization_likelihood(xⁿ, yⁿ; 
                R=R, EPOCHS=EPOCHS, M=M, P=P, n_iterations=n_iterations, m₁=m₁, m₂=m₂)
            parameters[pair[:id]][R]['<'], likelihoods[pair[:id]][R]['<'] = bayesian_optimization_likelihood(xⁿ, yⁿ; 
                R=R, EPOCHS=EPOCHS, M=M, P=P, n_iterations=n_iterations, m₁=m₁, m₂=m₂)
        end        
    end
    metaparameters = Dict("M"=>M, "EPOCHS"=>EPOCHS, "P"=>P, "m₁"=>m₁, "m₂"=>m₂, "T_max"=>T_max, 
        "Rs"=>Rs, "warm_start_type"=>warm_start_type, "n_iterations"=>n_iterations, 
        "SEED"=>SEED, "norm"=>norm)
    save_json("$RESULTS_PATH/$(name)_$(warm_start_type)/metaparameters.json", metaparameters=metaparameters)
    save_json("$RESULTS_PATH/$(name)_$(warm_start_type)/parameters.json", parameters=parameters);
    save_json("$RESULTS_PATH/$(name)_$(warm_start_type)/likelihoods.json", likelihoods=likelihoods);
    end;

In [4]:
vb_hyperparameter_optimization(tuebingen_data[51:75],1000; name="hp_opt_likelihood_3", M=4, P=100, Rs=1:5, m₁=0.0, m₂=0.0,
        EPOCHS=200, warm_start_type="free", n_iterations=200)

pair0051...			 1


┌ Warning: `lgamma(x::Real)` is deprecated, use `(logabsgamma(x))[1]` instead.
│   caller = #v_bayes#24(::Int64, ::Float64, ::Float64, ::Float64, ::Float64, ::Float64, ::Float64, ::Float64, ::Float64, ::Float64, ::Int64, ::typeof(v_bayes), ::Array{Float64,2}, ::Int64) at BayesianCausality.jl:441
└ @ Main.BayesianCausality /home/melih/Desktop/projects/aistats_causality/src/BayesianCausality.jl:441
┌ Warning: `lgamma(x::Real)` is deprecated, use `(logabsgamma(x))[1]` instead.
│   caller = #v_bayes#24(::Int64, ::Float64, ::Float64, ::Float64, ::Float64, ::Float64, ::Float64, ::Float64, ::Float64, ::Float64, ::Int64, ::typeof(v_bayes), ::Array{Float64,2}, ::Int64) at BayesianCausality.jl:443
└ @ Main.BayesianCausality /home/melih/Desktop/projects/aistats_causality/src/BayesianCausality.jl:443
┌ Warning: `lgamma(x::Real)` is deprecated, use `(logabsgamma(x))[1]` instead.
│   caller = #v_bayes#24(::Int64, ::Float64, ::Float64, ::Float64, ::Float64, ::Float64, ::Float64, ::Float64, ::Float64,

 27.628066 seconds (3.89 M allocations: 269.692 MiB, 0.39% gc time)
 24.472834 seconds (3.23 M allocations: 359.076 MiB, 0.43% gc time)
 22.509502 seconds (3.23 M allocations: 359.076 MiB, 0.46% gc time)
 2
 71.107539 seconds (171.23 M allocations: 12.559 GiB, 4.95% gc time)
106.457625 seconds (264.69 M allocations: 28.734 GiB, 7.83% gc time)
104.376894 seconds (271.77 M allocations: 29.508 GiB, 7.65% gc time)
 3
103.534701 seconds (347.96 M allocations: 25.637 GiB, 6.55% gc time)
135.135163 seconds (400.77 M allocations: 44.040 GiB, 7.93% gc time)
235.947877 seconds (711.38 M allocations: 78.321 GiB, 8.66% gc time)
 4
145.177612 seconds (556.15 M allocations: 40.921 GiB, 7.47% gc time)
186.050763 seconds (582.67 M allocations: 64.031 GiB, 9.55% gc time)
207.801239 seconds (631.73 M allocations: 69.599 GiB, 9.86% gc time)
 5
- 06:57:39.484861 ERROR: nlopt failure
 73.170519 seconds (328.87 M allocations: 24.246 GiB, 9.62% gc time)
234.839642 seconds (764.83 M allocations: 84.292 GiB, 1

 78.188756 seconds (159.93 M allocations: 12.198 GiB, 5.37% gc time)
107.743012 seconds (201.70 M allocations: 22.164 GiB, 7.23% gc time)
 93.209622 seconds (170.70 M allocations: 18.716 GiB, 6.96% gc time)
 4
108.942280 seconds (264.02 M allocations: 20.071 GiB, 6.52% gc time)
133.866731 seconds (270.23 M allocations: 29.815 GiB, 7.19% gc time)
117.818823 seconds (232.12 M allocations: 25.547 GiB, 6.00% gc time)
 5
- 09:27:19.948174 ERROR: nlopt failure
 73.380158 seconds (205.91 M allocations: 15.661 GiB, 6.68% gc time)
137.875683 seconds (296.76 M allocations: 32.744 GiB, 7.17% gc time)
156.893908 seconds (356.07 M allocations: 39.307 GiB, 7.62% gc time)
pair0063...			 1
 22.918050 seconds (1.72 M allocations: 137.886 MiB, 0.14% gc time)
 24.215449 seconds (1.89 M allocations: 209.347 MiB, 0.21% gc time)
 21.760547 seconds (1.89 M allocations: 209.347 MiB, 0.23% gc time)
 2
 51.377652 seconds (80.17 M allocations: 6.120 GiB, 3.62% gc time)
 60.104354 seconds (72.92 M allocations: 7.

 25.394454 seconds (8.16 M allocations: 908.432 MiB, 0.94% gc time)
 24.573145 seconds (8.16 M allocations: 908.432 MiB, 0.88% gc time)
 2
 29.477213 seconds (38.09 M allocations: 2.707 GiB, 2.56% gc time)
 28.436357 seconds (33.47 M allocations: 3.609 GiB, 3.29% gc time)
 34.299462 seconds (41.69 M allocations: 4.503 GiB, 3.21% gc time)
 3
 60.109710 seconds (265.84 M allocations: 19.033 GiB, 8.72% gc time)
 86.690200 seconds (273.75 M allocations: 30.153 GiB, 9.65% gc time)
163.584295 seconds (648.37 M allocations: 71.509 GiB, 12.20% gc time)
 4
 59.996118 seconds (270.54 M allocations: 19.381 GiB, 8.96% gc time)
233.141737 seconds (991.93 M allocations: 109.927 GiB, 14.00% gc time)
230.041165 seconds (965.08 M allocations: 106.806 GiB, 14.29% gc time)
 5
 79.959608 seconds (448.70 M allocations: 32.253 GiB, 11.44% gc time)
181.827285 seconds (773.65 M allocations: 85.755 GiB, 13.57% gc time)
106.742016 seconds (386.68 M allocations: 42.950 GiB, 11.73% gc time)
pair0072...			 1
 21.1

198.688591 seconds (789.24 M allocations: 86.764 GiB, 10.71% gc time)
 4
201.751803 seconds (1.24 G allocations: 89.332 GiB, 11.09% gc time)
257.030791 seconds (1.07 G allocations: 118.463 GiB, 11.95% gc time)
318.682771 seconds (1.35 G allocations: 149.100 GiB, 12.84% gc time)
 5
268.016326 seconds (1.69 G allocations: 122.014 GiB, 11.83% gc time)
332.483782 seconds (1.60 G allocations: 176.889 GiB, 12.23% gc time)
295.083214 seconds (1.37 G allocations: 151.934 GiB, 12.78% gc time)
pair0079...			 1
 19.246260 seconds (5.39 M allocations: 402.402 MiB, 0.47% gc time)
 20.653155 seconds (6.00 M allocations: 667.103 MiB, 0.67% gc time)
 24.749410 seconds (6.00 M allocations: 667.103 MiB, 0.57% gc time)
 2
 54.939477 seconds (243.14 M allocations: 17.378 GiB, 7.59% gc time)
124.953201 seconds (476.43 M allocations: 51.726 GiB, 9.94% gc time)
 90.329068 seconds (333.42 M allocations: 36.168 GiB, 9.86% gc time)
 3
111.055830 seconds (652.74 M allocations: 47.025 GiB, 10.62% gc time)
- 19:35

"{\"likelihoods\":{\"pair0070\":{\"4\":{\"<\":-3098.7246326518425,\"^\":-2597.2898841286183,\">\":-3354.175735588416},\"2\":{\"<\":-3819.7945809096145,\"^\":-3636.244134082496,\">\":-4330.030406628018},\"3\":{\"<\":-2218.294770124093,\"^\":-3421.0779050948227,\">\":-3747.324246543278},\"5\":{\"<\":-3598.698753222878,\"^\":-3857.062044896515,\">\":-4353.278291572717},\"1\":{\"<\":2611.34641857826,\"^\":2843.0369304419182,\">\":2607.4695778437463}},\"pair0080\":{\"4\":{\"<\":1306.0668158786495,\"^\":1318.7108951369419,\">\":1343.311239329921},\"2\":{\"<\":1390.5608693268778,\"^\":1529.7329650606034,\">\":1393.670789024243},\"3\":{\"<\":1306.658707732379,\"^\":1410.1933707748283,\">\":1314.6495587971035},\"5\":{\"<\":1325.6879701893665,\"^\":1321.7175015038656,\">\":1372.908752673333},\"1\":{\"<\":1897.7543471655672,\"^\":2045.2142835588818,\">\":1897.0474834245595}},\"pair0063\":{\"4\":{\"<\":534.2763222200558,\"^\":518.6792425124354,\">\":527.527877030838},\"2\":{\"<\":518.3061244510103